In [ ]:
# Initial imports
import panel as pn
pn.extension('plotly')

import datetime as dt
import hvplot.pandas
import json
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import requests
import seaborn as sns
import time

from pathlib import Path
from dotenv import load_dotenv

%matplotlib inline

## Part 1: Trading Volume Data

Coinbase provides historical data for their products via their [public API](https://docs.pro.coinbase.com). For the trading volume analysis, the following endpoints are queried: `/currencies` and `/products`.

In [ ]:
# Coinbase API endpoints
coinbase_api_url = "https://api.pro.coinbase.com"
cb_products_api_url = f"{coinbase_api_url}/products"
cb_currencies_api_url = f"{coinbase_api_url}/currencies"

In [ ]:
# Get currencies (fiat and crypto)
cb_currencies_response = requests.get(cb_currencies_api_url)
cb_currencies = cb_currencies_response.json()
cb_currencies_df = pd.DataFrame(cb_currencies)
cb_currencies_df = pd.concat([cb_currencies_df, pd.io.json.json_normalize(cb_currencies_df["details"])], axis=1).drop(columns=["details"])
cb_currencies_df.head()

In [ ]:
# Get Coinbase products
cb_products_response = requests.get(cb_products_api_url)
cb_products = cb_products_response.json()
cb_products_df = pd.DataFrame(cb_products)
cb_products_df.head()

In [ ]:
def get_date_list(start_date, end_date, interval=300):
    """
    Get the list of dates in 300-day incremenets.
    """
    delta_days = (end_date - start_date).days
    iterations = math.floor(delta_days / interval) + 1
    date_list = [start_date + dt.timedelta(days=x*interval) for x in range(iterations)]
    return date_list

In [ ]:
def get_coinbase_historical_data(id, start_date):
    """
    Get historical data (daily) for a product between the start and end dates.
    """
    start_str = dt.datetime.strftime(start_date, "%Y-%m-%d")
    end_str = dt.datetime.strftime(start_date + dt.timedelta(days=300), "%Y-%m-%d")
    candles_url = f"{cb_products_api_url}/{id}/candles?start={start_str}&end={end_str}&granularity=86400"
    candles_response = requests.get(candles_url)
    candles = candles_response.json()
    candles_df = pd.DataFrame(candles, columns=["time", "low", "high", "open", "close", "volume"])
    return candles_df

In [ ]:
# Get list of fiat and cryptocurrencies
cb_crypto_currencies = cb_currencies_df[cb_currencies_df["type"] == "crypto"]["id"].tolist()
cb_fiat_currencies = cb_currencies_df[cb_currencies_df["type"] == "fiat"]["id"].tolist()
# Find Coinbase products bought/sold with fiat currency
cb_fiat_products_df = cb_products_df[cb_products_df["quote_currency"].isin(cb_fiat_currencies)]

print(f"Crypto currencies: {cb_crypto_currencies}\n")
print(f"Fiat currencies: {cb_fiat_currencies}\n")
print(f"{len(cb_fiat_products_df.index)} Coinbase products bought/sold with fiat currency")

In [ ]:
# Get list of dates to fetch data for in 300-day increments
today = dt.datetime.today()
base = dt.datetime.strptime("2021-01-01", "%Y-%m-%d")
cb_date_list = get_date_list(base, today)

# Get data from Coinbase and append to dataframe
trading_volume_data_df = pd.DataFrame()
for idx, row in cb_fiat_products_df.iterrows():
    product_id = row["id"]
    base_currency = row["base_currency"]
    quote_currency = row["quote_currency"]
    for start_date in cb_date_list:
        historical_df = get_coinbase_historical_data(product_id, start_date)
        if not historical_df.empty:
            historical_df["base_currency"] = base_currency
            historical_df["quote_currency"] = quote_currency
            trading_volume_data_df = trading_volume_data_df.append(historical_df)
        time.sleep(2)

trading_volume_data_df.head()

In [ ]:
# convert epoch time to date
trading_volume_data_df = trading_volume_data_df.drop_duplicates()
trading_volume_data_df["date"] = pd.to_datetime(trading_volume_data_df["time"], unit="s")
trading_volume_data_df = trading_volume_data_df.drop(columns=["time"])

In [ ]:
# Export dataframe to CSV for faster loading
cb_data_filepath = Path("../data/coinbase_historical_data1.csv")
trading_volume_data_df.to_csv(cb_data_filepath, index=False)